In [1]:
import LowLevelFEM as FEM
using LowLevelFEM

gmsh.initialize()

In [2]:
gmsh.open("test-1.geo")

Info    : Reading 'test-1.geo'...
Info    : Reconstructing periodicity for curve connection 4 - 2
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 30%] Meshing curve 2 (Line)
Info    : [ 60%] Meshing curve 3 (Line)
Info    : [ 80%] Meshing curve 4 (Line) as a copy of curve 2
Info    : Done meshing 1D (Wall 0.00188742s, CPU 0.001888s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Plane, Frontal-Delaunay)
Info    : Done meshing 2D (Wall 0.00561051s, CPU 0.005592s)
Info    : Reconstructing periodicity for curve connection 4 - 2
Info    : 44 nodes 90 elements
Info    : Done reading 'test-1.geo'


In [3]:
#gmsh.fltk.run()

In [4]:
mat = FEM.material("body", E=3e5, ν=0.2, k=1, α=8e-6)
#problem = FEM.Problem([mat], type="PlaneHeatConduction", bandwidth="none")
problem = FEM.Problem([mat], type="AxiSymmetricHeatConduction", bandwidth="none")

Info    : RCMK renumbering...
Info    : Done RCMK renumbering (bandwidth is now 10)


LowLevelFEM.Problem("test-1", "AxiSymmetricHeatConduction", 2, 1, LowLevelFEM.Material[LowLevelFEM.Material("body", 300000.0, 0.2, 7.85e-9, 1.0, 4.2e8, 8.0e-6)], 1.0, 44)

In [5]:
K = FEM.heatConductionMatrix(problem)

44×44 SparseArrays.SparseMatrixCSC{Float64, Int64} with 262 stored entries:
⎡⠑⢄⠁⠐⠂⠠⠄⢀⡀⠈⠀⠀⠀⠀⠀⠀⠀⠀⠀⢀⠡⠂⎤
⎢⢁⠀⠻⣦⠀⠀⠀⠀⠀⠀⠀⠰⡄⠀⠀⠃⠀⠀⠀⠀⠁⡀⎥
⎢⠈⡀⠀⠀⠻⣦⠀⠀⠀⠀⠆⠀⠀⠀⠀⠀⠀⠘⡄⠀⢀⠁⎥
⎢⠀⢁⠀⠀⠀⠀⠻⣦⠀⠀⠀⠆⠀⠀⢠⠀⠀⠃⠀⢀⠈⠀⎥
⎢⡀⠈⠀⠀⠀⠀⠀⠀⠻⣦⠰⠀⠀⠀⠀⠘⢠⠀⠀⠈⡀⠀⎥
⎢⠀⠀⢀⡀⠈⠁⠠⠄⠐⠂⠑⢄⣈⡈⠥⡐⠒⠌⠡⠂⠀⠀⎥
⎢⠀⠀⠀⠉⠀⠀⠀⠀⠀⠀⡂⠸⢻⣶⡀⠄⠀⠘⢀⠄⠀⢡⎥
⎢⠀⠀⠤⠀⠀⠀⠀⠒⣀⠀⢁⠣⠀⠌⢑⢔⡢⠁⠙⢔⠄⠀⎥
⎢⠀⠀⠀⠀⣀⠀⠤⠀⠀⠒⡘⠄⣀⠀⠌⠊⠑⢄⠌⠃⠢⡈⎥
⎢⠀⢀⠀⠀⠀⠉⠀⢀⡀⠀⠡⠂⠀⠔⢓⢄⠦⠁⠑⢄⠈⠰⎥
⎣⠡⠂⠁⠠⠄⠐⠂⠀⠀⠈⠀⠀⠄⣀⠀⠁⡈⠢⢂⡀⠑⢄⎦

In [6]:
#fqn(x, y, z) = 10

In [7]:
#right = FEM.temperatureConstraint("right", T=0)
right = FEM.heatConvection("right", h=1, Tₐ=20)
left = FEM.heatFlux("left", qn=-1);

In [8]:
q = FEM.heatFluxVector(problem, [left]);

In [9]:
FEM.applyHeatConvection!(problem, K, q, [right]);

In [10]:
#FEM.applyBoundaryConditions!(problem, K, q, [right])

In [11]:
T = FEM.solveTemperature(K, q)

44-element Vector{Float64}:
 89.57604719204086
 20.49723853878653
 20.4972332473231
 89.56879196388932
 68.72730196129812
 52.68066049265491
 39.85321183064054
 29.319788484910006
 20.50348360583522
 20.49774815933616
  ⋮
 28.407395034622365
 27.714707078337543
 49.74359756847307
 63.05890948685673
 75.78085444831679
 76.15350775890259
 26.810352008956365
 26.81790127553062
 54.02417084005417

In [12]:
Q = FEM.solveHeatFlux(problem, T);

In [13]:
Q2 = FEM.solveHeatFlux(problem, T, DoFResults=true)

88×1 Matrix{Float64}:
  0.9095075750699066
  0.007974379362560402
  0.5196012143129602
  0.0021133344594331804
  0.519525375037177
 -0.0020701348910341366
  0.9085291862530954
 -0.008122842180218415
  0.817392459557654
  0.007039498940038784
  ⋮
  0.0002466385207982258
  0.8744208943496986
 -0.00013340972906528137
  0.5321458672748611
 -1.3429099421657931e-5
  0.5328325186404279
  1.6029365409053752e-5
  0.6994248324782678
  0.0008733324586931435

In [14]:
T1 = FEM.showDoFResults(problem, T, "T", name="T", visible=true)

1

In [15]:
q1 = FEM.showHeatFluxResults(problem, Q, "qvec", visible=false, smooth=false)
qx = FEM.showHeatFluxResults(problem, Q, "q", visible=false, smooth=true)
q2 = FEM.showDoFResults(problem, Q2, "uvec")

Info    : Running Plugin(Smooth)...
Info    : Done running Plugin(Smooth)


4

In [16]:
FEM.resultant(problem, qx, "left")

57963.48563273149

In [17]:
FEM.resultant(problem, qx, "right")

65271.23315794376

In [18]:
Qx = FEM.resultant(q, "left", dim=1)

62831.85307179586

In [19]:
p1 = FEM.plotOnPath(problem, "bottom", qx)

5

In [20]:
fh0 = FEM.heatConvectionVector(problem, [right])
fh = FEM.showDoFResults(problem, fh0, "uvec", name="fh", visible=false)

6

In [21]:
gmsh.fltk.run()

-------------------------------------------------------
Version       : 4.13.1
License       : GNU General Public License
Build OS      : Linux64-sdk
Build date    : 19700101
Build host    : amdci7.julia.csail.mit.edu
Build options : 64Bit ALGLIB[contrib] ANN[contrib] Bamg Blossom Cairo DIntegration Dlopen DomHex Eigen[contrib] Fltk GMP Gmm[contrib] Hxt Jpeg Kbipack LinuxJoystick MathEx[contrib] Mesh Metis[contrib] Mmg Mpeg Netgen Nii2mesh ONELAB ONELABMetamodel OpenCASCADE OpenCASCADE-CAF OpenGL OpenMP OptHom Parser Plugins Png Post QuadMeshingTools QuadTri Solver TetGen/BR TinyXML2[contrib] Untangle Voro++[contrib] WinslowUntangler Zlib
FLTK version  : 1.3.8
OCC version   : 7.7.2
Packaged by   : root
Web site      : https://gmsh.info
Issue tracker : https://gitlab.onelab.info/gmsh/gmsh/issues
-------------------------------------------------------


XOpenIM() failed
XRequest.18: BadValue 0x0


In [22]:
#problem2 = FEM.Problem([mat], type="AxiSymmetric", bandwidth="none")

In [23]:
#K2 = FEM.stiffnessMatrix(problem2)

In [24]:
#supp = FEM.displacementConstraint("support", uy=0)
#load = FEM.load("support", fx=0);

In [25]:
#f = FEM.loadVector(problem2, [load]);

In [26]:
#fT = FEM.thermalLoadVector(problem2, T);

In [27]:
#f2 = f + fT;

In [28]:
#FEM.applyBoundaryConditions!(problem2, K2, f2, [supp])

In [29]:
#q2 = FEM.solveDisplacement(K2, f2);

In [30]:
#E = FEM.solveStrain(problem2, q2);

In [31]:
#S = FEM.solveStress(problem2, q2, T=T);

In [32]:
#u = FEM.showDoFResults(problem2, q2, "uvec", visible=false)
#ux = FEM.showDoFResults(problem2, q2, "ux", visible=false)
#uy = FEM.showDoFResults(problem2, q2, "uy", visible=false)

In [33]:
#ex = FEM.showStrainResults(problem2, E, "ex", visible=false)
#ey = FEM.showStrainResults(problem2, E, "ey", visible=false)
#exy = FEM.showStrainResults(problem2, E, "exy", visible=false)

In [34]:
#seqv = FEM.showStressResults(problem2, S, "seqv", visible=false)
#sx = FEM.showStressResults(problem2, S, "sx", visible=false)
#sy = FEM.showStressResults(problem2, S, "sy", visible=false)
#sxy = FEM.showStressResults(problem2, S, "sxy", visible=false)

In [35]:
#FF = FEM.showDoFResults(problem2, fT, "uvec", name="fT", visible=false)

In [36]:
#gmsh.fltk.run()

In [37]:
gmsh.finalize()